In [2]:
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
from scipy.optimize import curve_fit
import datetime
from matplotlib.dates import date2num
import numpy as np

raw_data = pd.read_csv('pacienti.csv', sep=r'\s*,\s*', parse_dates=[0], dayfirst=True)

c:\users\sorin\anaconda3\envs\covid19\lib\site-packages\ipykernel_launcher.py:9: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.



In [6]:
date = pd.date_range(start='26-02-2020', periods=20).to_frame(name='data')
infectii_interne = raw_data[raw_data['tip_infectie']=='I'].groupby('data').size().reset_index(name='infectii_interne')
infectii_externe = raw_data[raw_data['tip_infectie']=='E'].groupby('data').size().reset_index(name='infectii_externe')
infectii = date.join(infectii_interne.set_index('data'), on='data').join(infectii_externe.set_index('data'), on='data')
infectii = infectii.fillna(0)
infectii["pe_zi"] = infectii["infectii_interne"] + infectii["infectii_externe"]
infectii = infectii.join(infectii["pe_zi"].cumsum().to_frame(name='cumulat'))
infectii

,data,infectii_interne,infectii_externe,pe_zi,cumulat
2020-02-26,2020-02-26,0.0,1.0,1.0,1.0
2020-02-27,2020-02-27,0.0,0.0,0.0,1.0
2020-02-28,2020-02-28,0.0,2.0,2.0,3.0
2020-02-29,2020-02-29,0.0,0.0,0.0,3.0
2020-03-01,2020-03-01,0.0,0.0,0.0,3.0
2020-03-02,2020-03-02,0.0,0.0,0.0,3.0
2020-03-03,2020-03-03,0.0,1.0,1.0,4.0
2020-03-04,2020-03-04,1.0,1.0,2.0,6.0
2020-03-05,2020-03-05,1.0,0.0,1.0,7.0
2020-03-06,2020-03-06,0.0,2.0,2.0,9.0


In [12]:
predictii = pd.read_csv('predictii.csv', sep=r'\s*,\s*', parse_dates=[0], dayfirst=True)

c:\users\sorin\anaconda3\envs\covid19\lib\site-packages\ipykernel_launcher.py:1: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.



In [13]:
def func(x, a, b, c):
    return a * np.exp(b * x) + c

l = len(infectii)
x = np.linspace(1, l, l)
y = list(infectii["infectii_interne"])

popt, pcov = curve_fit(func, x, y)

In [14]:
#Cazuri interne
inf_pred = infectii.merge(predictii, left_on='data', right_on='data', how='outer')
x = inf_pred['data']
y = inf_pred['infectii_interne']

x2 = np.linspace(1, l+1, l+1)

fig = go.Figure(data=[
    go.Scatter(name='Predictie', x=x, y=func(x2, *popt)),
    go.Bar(name='Real', x=x, y=y)
])

fig.update_layout(title="Infectii noi pe zi - transmise in Romania",
    yaxis=dict(range=[-1, 30]))

#py.iplot(fig)
fig.show()
#pio.write_html(fig, file='infectii_noi_zi_romania.html', auto_open=True)

In [15]:
#func(i, *popt) for i in range()
sum([func(i, *popt) for i in range(19, 33)]) + 139.0

766.9896016440223

In [16]:
predictii
infectii

,data,infectii_interne,infectii_externe,pe_zi,cumulat
2020-02-26,2020-02-26,0.0,1.0,1.0,1.0
2020-02-27,2020-02-27,0.0,0.0,0.0,1.0
2020-02-28,2020-02-28,0.0,2.0,2.0,3.0
2020-02-29,2020-02-29,0.0,0.0,0.0,3.0
2020-03-01,2020-03-01,0.0,0.0,0.0,3.0
2020-03-02,2020-03-02,0.0,0.0,0.0,3.0
2020-03-03,2020-03-03,0.0,1.0,1.0,4.0
2020-03-04,2020-03-04,1.0,1.0,2.0,6.0
2020-03-05,2020-03-05,1.0,0.0,1.0,7.0
2020-03-06,2020-03-06,0.0,2.0,2.0,9.0


In [17]:
#Total cazuri
inf_pred = infectii.merge(predictii, left_on='data', right_on='data', how='outer')
x = inf_pred['data']
y1 = inf_pred['cumulat']
y2 = inf_pred['predictie_cumulat_t1']

fig = go.Figure(data=[
    go.Bar(name='Predictie', x=x, y=y2),
    go.Bar(name='Real', x=x, y=y1)
])

fig.update_layout(title="Numar total de infectii")

#py.iplot(fig)
fig.show()
#pio.write_html(fig, file='infectii_totale.html', auto_open=True)

In [18]:
#Cazuri externe pe zi
x = inf_pred['data']
y = inf_pred['infectii_externe']

fig = go.Figure(data=[
    go.Bar(name='LA Zoo', x=x, y=y)
])
# Change the bar mode
fig.update_layout(title="Infectii noi pe zi - la persoane venite din strainatate",
    yaxis=dict(range=[-1, 30]))
#fig.show()

import plotly
import chart_studio
import chart_studio.plotly as py
from plotly.offline import plot, iplot

#chart_studio.tools.set_credentials_file(username='sorin.bolos', api_key='1VPkxbMHak7Y7M9dyDQD')
py.iplot(fig)
#pio.write_html(fig, file='infectii_noi_zi_externe.html', auto_open=True)

In [19]:
#Infectii totale
x = inf_pred['data']
y1 = inf_pred['infectii_externe']
y2 = inf_pred['infectii_interne']

fig = go.Figure(data=[
    go.Bar(name='Infectii la persoane revenite in tara', x=x, y=y1),
    go.Bar(name='Infectii realizate in Romania', x=x, y=y2)
])
# Change the bar mode
fig.update_layout(barmode='stack',title="Infectii pe zi")
#py.iplot(fig)
fig.show()
#pio.write_html(fig, file='infectii_noi_zi.html', auto_open=True)

In [30]:
def sigmoid(x, x0,y0,c,k):
    return c / (1 + np.exp(-k*(x-x0))) + y0

def func(x, a, b, c):
    return a * np.exp(b * x) + c

l = len(infectii)
x_s = np.linspace(1, l, l)
y_s = list(i/1000 for i in infectii["cumulat"])

popt_s, pcov_s = curve_fit(sigmoid, x_s, y_s)

xp = infectii['data']
yp = infectii['cumulat']

x2 = np.linspace(1, l+1, l+1)

fig = go.Figure(data=[
    go.Scatter(name='SF Zoo', x=x2, y=1000*sigmoid(x2, *popt_s)),
    go.Bar(name='LA Zoo', x=x2, y=yp)
])
# Change the bar mode
fig.update_layout()
fig.show()
popt_s

array([1.75969409e+01, 3.28741140e-03, 2.08257173e-01, 5.33025354e-01])

In [38]:
1000*sigmoid(34, *popt_s)

211.51136561965464

In [43]:
infectii[["data", "pe_zi", "cumulat"]].to_csv('pacienti_agg.csv', index=False)